<a href="https://colab.research.google.com/github/skywalker0803r/works/blob/master/%E8%BC%95%E6%B2%B9%E6%A1%88%E5%AD%90%E6%96%B0%E6%B8%AC%E8%A9%A6%E8%B3%87%E6%96%99%E6%A8%A1%E5%9E%8B%E8%A1%A8%E7%8F%BE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
model_dir_path = '/content/drive/My Drive/台塑輕油案子/S_F_L_SVR_models/'
test1_path = '/content/drive/My Drive/台塑輕油案子/test_without_outlier.csv'
test2_path = '/content/drive/My Drive/台塑輕油案子/Aroma-1 Import Naphtha Lab from Ship_Summary.xlsx'

# load model

In [0]:
from sklearn.base import TransformerMixin
from sklearn.preprocessing import PolynomialFeatures

class featuresGenerator(TransformerMixin):
  def __init__(self):#初始化建立一個交互項產生器,只兩兩相乘,不包括bias
    self.model = PolynomialFeatures(2,interaction_only=True,include_bias=False)
  
  def fit(self,X,y):
    return self
  
  def transform(self,X): 
    return  self.model.fit_transform(X) 

In [0]:
from sklearn.linear_model import Lasso
import numpy as np

class LassoTransformer(TransformerMixin):  
  def __init__(self):#初始化首先建立一個Lasso model
    self.model = Lasso()
  
  def set_params(self,alpha):#設置alpha參數會影響選擇變數多寡
    self.model.set_params(alpha=alpha)
  
  def fit(self,X,y):#對資料做訓練得到coef_
    self.model.fit(X,y)
    return self
  
  def transform(self,X):#呼叫此方法可過濾掉冗餘特徵
    SelectColumn = (abs(self.model.coef_)!=0)
    return X[:,SelectColumn]

In [0]:
import os
import joblib
models = {}
for i in os.listdir(model_dir_path):
  filename = model_dir_path+i
  i = i.split('.')[0].replace('_','')
  models[i] = joblib.load(filename)

In [6]:
models.keys()

dict_keys(['C5NP', 'C5IP', 'C5N', 'C6NP', 'C6IP', 'C6N', 'C6A', 'C7A', 'C7NP', 'C8NP', 'C7IP', 'C8IP', 'C7N', 'C8N', 'C9NP', 'C8A', 'C9IP', 'C9N', 'C9A', 'C10NP', 'C10IP', 'C10N', 'C10A'])

# load test1

In [7]:
import pandas as pd
test1 = pd.read_csv(test1_path)
new_col_name = ['T10','T50','T90','NA']+[i.upper().replace('_','').replace('-','') for i in test1.iloc[:,4:].columns.tolist()]
test1.columns = new_col_name
test1.head()

,T10,T50,T90,NA,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,98.6,119.2,146.1,40.133,1.142,0.616,0.217,3.745,2.577,4.028,0.557,7.669,5.9900,10.206,3.075,7.920,8.036,7.039,5.438,5.263,9.537,4.877,3.481,1.088,5.652,0.618,0.597
1,105.4,122.2,148.2,32.175,0.122,0.078,0.029,3.955,1.563,2.407,0.400,10.016,7.2633,6.573,2.809,9.544,9.815,5.135,5.482,6.793,10.850,4.308,3.963,1.069,6.025,0.585,0.484
2,105.8,122.7,149.2,31.428,0.098,0.064,0.020,3.920,1.447,2.296,0.396,10.236,7.3480,6.259,2.797,9.655,9.986,4.942,5.453,6.947,10.996,4.269,3.997,1.034,6.107,0.543,0.456
3,105.4,122.4,147.6,31.390,0.122,0.085,0.021,3.900,1.437,2.274,0.397,10.259,7.3560,6.234,2.820,9.670,10.146,4.928,5.476,6.953,11.026,4.264,3.982,1.013,6.054,0.545,0.449
4,105.4,122.6,148.2,32.418,0.072,0.049,0.015,3.873,1.369,2.206,0.489,10.213,7.3190,6.009,2.968,9.564,10.052,4.769,6.584,6.866,10.892,4.169,4.227,0.930,5.828,0.519,0.463


In [0]:
X_test = test1.iloc[:,:4]
y_test = test1.iloc[:,4:]

In [0]:
y_pred = pd.DataFrame(index=y_test.index,columns=y_test.columns)

# 特徵工程

In [0]:
def 特徵工程(df):
  df['T90-T50'] = df['T90']-df['T50']
  df['T50-T10'] = df['T50']-df['T10']
  return df

In [11]:
X_test = 特徵工程(X_test)
X_test.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,T10,T50,T90,NA,T90-T50,T50-T10
0,98.6,119.2,146.1,40.133,26.9,20.6
1,105.4,122.2,148.2,32.175,26.0,16.8
2,105.8,122.7,149.2,31.428,26.5,16.9
3,105.4,122.4,147.6,31.390,25.2,17.0
4,105.4,122.6,148.2,32.418,25.6,17.2


# model預測

In [12]:
for y_name in models.keys():
  pred = models[y_name].predict(X_test)
  y_pred[y_name] = pred
y_pred.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7NP,C7IP,C7N,C7A,C8NP,C8IP,C8N,C8A,C9NP,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.191988,0.824389,0.165207,3.546049,2.305583,3.499468,0.617205,7.947754,6.293707,9.615315,3.264777,8.020495,8.373682,6.865212,5.891685,5.207565,9.273638,5.024542,3.408418,1.102844,5.059223,0.714474,0.662601
1,0.132381,0.086538,0.101532,3.360802,1.129945,2.168158,0.421458,10.406509,7.618275,6.327312,2.865747,9.652671,10.203371,4.979293,5.615177,6.684378,10.663944,4.417900,3.867578,1.199263,5.950572,0.617702,0.705440
2,0.119955,0.063436,0.102912,3.323328,1.114417,2.098173,0.396657,10.402525,7.605004,6.132867,2.777820,9.720654,10.261553,4.807384,5.497477,6.773678,10.716336,4.320622,3.926804,1.300674,6.117179,0.637208,0.872996
3,0.113011,0.066706,0.103271,3.564747,1.204040,2.121922,0.429815,10.330726,7.447542,6.048433,2.816378,9.819001,10.326989,4.839130,5.598420,6.848584,10.839803,4.323246,3.914898,1.143704,6.029713,0.572690,0.607114
4,0.156517,0.090036,0.101861,3.419551,1.174572,2.144542,0.423895,10.178394,7.400757,6.258235,2.858889,9.606877,10.128533,4.985365,5.718934,6.724423,10.745011,4.379373,3.964622,1.216983,6.052516,0.617922,0.717748


# 評分工具

In [0]:
from sklearn.metrics import r2_score
import numpy as np

def rmse(y_true, y_pred):
    return np.sqrt(((y_pred - y_true) ** 2).mean())

def mape(y_true, y_pred,dropinf=True):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    ape = np.abs((y_true - y_pred) / y_true)
    if dropinf == True:
      ape = ape[np.isfinite(ape)]
    return np.mean(ape) * 100

def metric(y_true,y_pred):
  R2 = r2_score(y_true,y_pred)
  RMSE = rmse(y_true,y_pred)
  MAPE = mape(y_true,y_pred)
  return {'R2':R2,'RMSE':RMSE,'MAPE':MAPE}

# test1結果

In [14]:
table = pd.DataFrame(index=y_test.columns,columns=['R2','RMSE','MAPE'])
for y_name in y_test.columns:
  table.loc[y_name] = metric(y_test[y_name],y_pred[y_name])
table.loc['AVG'] = table.mean()
table

,R2,RMSE,MAPE
C5NP,0.884713,0.290165,27.2194
C5IP,0.738117,0.261873,41.6517
C5N,0.78577,0.0633628,225.162
C6NP,0.0883964,0.386282,6.60118
C6IP,0.866548,0.381845,9.49235
C6N,0.879682,0.574429,5.50099
C6A,0.533583,0.116681,17.5323
C7NP,0.98063,0.306088,2.94169
C7IP,0.859847,0.413681,4.42108
C7N,0.980959,0.575045,3.40638


In [15]:
table[table.RMSE == table.RMSE.max()]

,R2,RMSE,MAPE
C7N,0.980959,0.575045,3.40638


#=============================================

# 載入test2

In [16]:
test2 = pd.read_excel(test2_path).iloc[42:,2:]
test2

,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,Unnamed: 35,Unnamed: 36,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,...,Unnamed: 105,Unnamed: 106,Unnamed: 107,Unnamed: 108,Unnamed: 109,Unnamed: 110,Unnamed: 111,Unnamed: 112,Unnamed: 113,Unnamed: 114,Unnamed: 115,Unnamed: 116,Unnamed: 117,Unnamed: 118,Unnamed: 119,Unnamed: 120,Unnamed: 121,Unnamed: 122,Unnamed: 123,Unnamed: 124,Unnamed: 125,Unnamed: 126,Unnamed: 127,Unnamed: 128,Unnamed: 129,Unnamed: 130,Unnamed: 131,Unnamed: 132,Unnamed: 133,Unnamed: 134,Unnamed: 135,Unnamed: 136,Unnamed: 137,Unnamed: 138,Unnamed: 139,Unnamed: 140,Unnamed: 141,Unnamed: 142,Unnamed: 143,Unnamed: 144
42,"N+A, wt%",NaN,52.2308,55.6277,40.6336,51.6905,37.1883,49.7165,51.8455,50.5755,53.0179,59.0963,52.6174,49.1129,58.8213,52.2533,60.9143,52.0662,62.3821,52.6004,61.1055,52.6538,52.0471,61.0706,51.8293,52.1912,60.6492,52.1335,55.0958,52.4067,45.2748,61.8589,40.993,61.363,54.1639,52.1211,35.0509,35.95,52.6795,35.8782,...,53.1268,50.5248,42.8068,53.406,58.2483,46.3966,52.7353,58.3005,57.7704,53.594,58.0097,57.8133,50.2114,55.6574,54.0002,59.4185,50.5124,53.3043,52.5137,41.9044,55.1133,49.1332,55.4487,39.0656,56.3991,61.0008,53.3492,41.5765,53.955,52.606,49.1608,40.3954,60.4946,41.7366,56.5521,56.4867,43.5138,47.1896,56.1377,49.8505
43,D86,"T10, oC",94.4,93.3,71.2,95.3,71.2,89,96.2,80.5,81.5,102,94.9,95.8,101.4,95.6,102.8,94.2,102.3,94,102.6,93,93.3,105.2,91.4,92.3,103.2,99.5,99.5,95,65.6,94.2,62.1,100.2,95.3,94.7,67,67.8,112.2,68.5,...,95.1,89.5,94.5,93.4,91.3,95,92.5,94.6,93.1,100.1,93.5,92.8,99.7,93.3,92.7,95.7,96.7,95.5,99.2,99.8,95.8,86.6,96.3,97.7,97,89.4,103.9,87.1,97.5,103.1,85.6,83.4,89.5,88,97.5,97.9,67.5,84.5,98.1,91.2
44,NaN,"T50, oC",113.1,107,91.6,116.1,106,109.4,116.1,111,101,114,112.9,112,112.5,114.1,110.7,113,109.4,112,109.5,111.6,113,113,111.4,111.6,110.9,119.8,125.5,114.3,99.6,108.2,95.8,110.5,115.1,114.6,110.2,109.8,130.2,109.4,...,114.7,112.6,115.8,113.9,107.7,113.3,112.8,115,115,113.3,113.5,113,121.2,113.3,113.3,111.7,117.2,115.5,114.4,128.2,115.4,109.9,116.2,123.3,116.5,105.2,114.5,111.4,117,113.8,108.4,118.1,103.7,109.6,117.3,118.1,94,107.7,117.9,112.9
45,NaN,"T90, oC",141.2,132.7,140.3,150.6,143.5,138.1,147.3,156.6,129.3,143.5,140.9,140.2,144.4,146.7,140.2,146.1,139.6,143,136,131.3,144.4,139.1,145.9,143.1,136.1,148.9,162.5,143.5,133.6,140.9,139,138.3,142.7,140.5,165,168.8,154.1,159.3,...,143.6,135.8,141.6,139.6,134,138,138.7,143,144.5,136.9,139.5,138.3,154,141,142.7,139.7,145.6,141.7,139.4,161.7,141.8,147.6,142.2,/148,141.6,134.9,136,148.5,142.7,134.5,144.3,152.6,132,154.5,145.3,147.8,126.8,142.9,146.3,147.7
46,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
47,PONA Composition,P5,2.26971,3.50654,8.25376,2.38986,11.5592,4.5402,2.31675,6.06856,6.61014,0.896857,2.20869,1.19403,0.70254,1.96776,0.0199976,2.39662,0.0040016,2.46056,0.055991,2.65158,3.01364,0.0439956,3.17452,2.87651,0.16797,2.74775,1.50282,2.36188,13.9308,3.18962,16.637,1.33284,2.56564,2.6,14.6699,13.6069,1.94363,13.8727,...,2.33372,4.36061,3.06975,2.72656,4.07755,2.98658,3.08563,2.52398,3.07706,1.327,2.984,3.0273,1.28001,2.71257,2.763,3.07203,2.092,2.312,1.33003,5.84292,2.112,2.98794,2.25345,3.81328,2.166,3.762,0.667,2.757,2,0.824,3.336,6.834,1.82304,2.36602,1.925,1.945,12.593,3.805,1.90296,1.907
48,NaN,N5,0.330957,0.245968,0.750887,0.310981,0.815869,0.40621,0.295967,0.420596,0.602922,0.116981,0.

In [17]:
NA = test2.iloc[0,2:]
T10 = test2.iloc[1,2:]
T50 = test2.iloc[2,2:]
T90 = test2.iloc[3,2:]
X_test = pd.DataFrame({'T10':T10,'T50':T50,'T90':T90,'NA':NA}).reset_index(drop=True)
X_test.head()

,T10,T50,T90,NA
0,94.4,113.1,141.2,52.2308
1,93.3,107,132.7,55.6277
2,71.2,91.6,140.3,40.6336
3,95.3,116.1,150.6,51.6905
4,71.2,106,143.5,37.1883


In [18]:
X_test.loc[124]

T10       97.7
T50      123.3
T90       /148
NA     39.0656
Name: 124, dtype: object

In [0]:
X_test.loc[124,'T90'] =148

# 特徵工程

In [0]:
X_test = 特徵工程(X_test)

# model預測

In [21]:
y_pred = pd.DataFrame()
for y_name in models.keys():
  pred = models[y_name].predict(X_test)
  y_pred[y_name] = pred
y_pred.head()

,C5NP,C5IP,C5N,C6NP,C6IP,C6N,C6A,C7A,C7NP,C8NP,C7IP,C8IP,C7N,C8N,C9NP,C8A,C9IP,C9N,C9A,C10NP,C10IP,C10N,C10A
0,1.716196,1.144071,0.250450,2.941815,2.750644,5.389329,0.577971,3.794711,5.698919,6.108636,5.416847,6.572140,17.468853,10.110021,3.141188,5.523314,7.181441,5.689297,2.455653,0.658813,3.657149,0.716289,0.572823
1,2.132945,1.389565,0.283585,1.677568,1.951550,5.438674,0.610817,5.142836,6.861612,4.368222,8.276494,5.974894,21.507773,9.195132,1.770988,4.329307,5.960378,4.925505,1.785919,0.627339,2.763656,0.569530,0.592016
2,3.804376,3.720335,0.737118,5.553909,6.551760,6.537489,3.117682,4.393829,5.982940,4.341361,6.237810,5.491561,9.375740,5.375424,2.617877,3.836463,4.642475,2.759120,1.722272,0.456031,2.371851,0.493243,0.599632
3,1.627218,1.124865,0.232292,2.880083,2.634397,5.394394,0.675763,3.664423,5.566625,5.784647,5.300208,6.036060,15.808896,9.769885,3.235077,5.317830,7.067578,5.279655,3.177994,1.477705,4.789737,1.136681,1.399677
4,6.747063,4.790403,0.653205,5.520083,5.936270,5.368607,1.011057,2.750870,6.463821,5.596671,6.566044,6.255897,9.780552,6.084521,3.183692,3.882773,6.621255,3.703517,2.082622,0.891317,3.376333,0.615212,0.773997


# 整理y_test

In [22]:
y_test = test2.loc[47:63,:].iloc[:,1:].T
y_test.columns = y_test.iloc[0].values
y_test = y_test.iloc[1:,:]
y_test.reset_index(drop=True,inplace=True)
y_test.head()

,P5,N5,P6,N6,A6,P7,N7,A7,P8,N8,A8,P9,N9,A9,P10,N10,A10
0,2.26971,0.330957,6.44416,6.50816,0.38595,9.38368,19.0036,3.1066,16.5558,9.17785,5.28631,8.64836,4.06374,2.83187,4.46746,0.536727,0.999033
1,3.50654,0.245968,3.37956,3.97748,0.517933,16.4447,26.1567,5.79525,12.2714,8.85283,3.93549,6.45364,3.39507,1.29384,2.31641,0.395516,1.06158
2,8.25376,0.750887,19.9868,10.3696,3.19552,15.4699,12.012,2.6706,7.45693,4.51527,2.0077,5.94249,2.20844,1.56262,2.25651,0.387673,0.953274
3,2.38986,0.310981,5.81864,5.89165,0.347979,8.62817,17.8342,2.94682,16.5347,8.8268,5.16869,8.72721,4.21492,3.22291,6.21095,0.795398,2.13011
4,11.5592,0.815869,12.1191,6.00304,0.957847,11.3209,10.2077,2.70757,13.177,5.71101,3.91137,11.1504,3.93581,2.31204,3.48531,0.262358,0.363675


In [23]:
for i in y_test.columns:
  y_test[i] = pd.to_numeric(y_test[i])
y_test.dtypes

P5     float64
N5     float64
P6     float64
N6     float64
A6     float64
P7     float64
N7     float64
A7     float64
P8     float64
N8     float64
A8     float64
P9     float64
N9     float64
A9     float64
P10    float64
N10    float64
A10    float64
dtype: object

# 整理y_pred

In [0]:
y_pred['P5'] = y_pred[['C5NP','C5IP']].sum(axis=1)
y_pred['N5'] = y_pred['C5N']

y_pred['P6'] = y_pred[['C6NP','C6IP']].sum(axis=1)
y_pred['N6'] = y_pred['C6N']
y_pred['A6'] = y_pred['C6A']

y_pred['P7'] = y_pred[['C7NP','C7IP']].sum(axis=1)
y_pred['N7'] = y_pred['C7N']
y_pred['A7'] = y_pred['C7A']

y_pred['P8'] = y_pred[['C8NP','C8IP']].sum(axis=1)
y_pred['N8'] = y_pred['C8N']
y_pred['A8'] = y_pred['C8A']

y_pred['P9'] = y_pred[['C9NP','C9IP']].sum(axis=1)
y_pred['N9'] = y_pred['C9N']
y_pred['A9'] = y_pred['C9A']

y_pred['P10'] = y_pred[['C10NP','C10IP']].sum(axis=1)
y_pred['N10'] = y_pred['C10N']
y_pred['A10'] = y_pred['C10A']

In [25]:
y_pred = y_pred.iloc[:,23:]
y_pred.head()

,P5,N5,P6,N6,A6,P7,N7,A7,P8,N8,A8,P9,N9,A9,P10,N10,A10
0,2.860267,0.250450,5.692459,5.389329,0.577971,11.115767,17.468853,3.794711,12.680776,10.110021,5.523314,10.322630,5.689297,2.455653,4.315962,0.716289,0.572823
1,3.522510,0.283585,3.629118,5.438674,0.610817,15.138106,21.507773,5.142836,10.343116,9.195132,4.329307,7.731366,4.925505,1.785919,3.390995,0.569530,0.592016
2,7.524710,0.737118,12.105669,6.537489,3.117682,12.220749,9.375740,4.393829,9.832922,5.375424,3.836463,7.260352,2.759120,1.722272,2.827881,0.493243,0.599632
3,2.752083,0.232292,5.514480,5.394394,0.675763,10.866832,15.808896,3.664423,11.820707,9.769885,5.317830,10.302655,5.279655,3.177994,6.267443,1.136681,1.399677
4,11.537466,0.653205,11.456353,5.368607,1.011057,13.029865,9.780552,2.750870,11.852568,6.084521,3.882773,9.804947,3.703517,2.082622,4.267651,0.615212,0.773997


# test2結果

In [26]:
table = pd.DataFrame(index=y_test.columns,columns=['R2','RMSE','MAPE'])
for y_name in y_test.columns:
  table.loc[y_name] = metric(y_test[y_name],y_pred[y_name])
table.loc['AVG'] = table.mean()
table

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: divide by zero encountered in true_divide
  if __name__ == '__main__':


,R2,RMSE,MAPE
P5,0.827391,1.69522,287.644
N5,0.148415,0.510435,117.826
P6,0.831693,1.44425,157.833
N6,0.515236,1.16086,14.3593
A6,-0.235737,0.914149,126.1
P7,0.663784,1.34795,10.0341
N7,0.598016,3.31368,14.5609
A7,0.467662,0.986551,22.6848
P8,0.333364,2.66529,17.4517
N8,0.376135,2.53787,23.1066
